In [32]:
#wikibook
!wget https://dumps.wikimedia.org/jawikibooks/20240320/jawikibooks-20240320-pages-articles-multistream.xml.bz2
!bzip2 -d jawikibooks-20240320-pages-articles-multistream.xml.bz2

--2024-04-09 03:23:35--  https://dumps.wikimedia.org/jawikibooks/20240320/jawikibooks-20240320-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23257442 (22M) [application/octet-stream]
Saving to: ‘jawikibooks-20240320-pages-articles-multistream.xml.bz2’

jawikibooks-2024032 100%[===================>]  22.18M  3.33MB/s    in 6.2s    

2024-04-09 03:23:41 (3.56 MB/s) - ‘jawikibooks-20240320-pages-articles-multistream.xml.bz2’ saved [23257442/23257442]

bzip2: Output file jawikibooks-20240320-pages-articles-multistream.xml already exists.


In [33]:
path="jawikibooks-20240320-pages-articles-multistream.xml"

In [34]:
import xml.etree.ElementTree as ET

# XMLファイルを読み込む
tree = ET.parse(path)
root = tree.getroot()

In [35]:
print(root.tag)

{http://www.mediawiki.org/xml/export-0.10/}mediawiki


In [36]:
cnt=0
for child in root:
    cnt+=1
    print(child.tag, child.attrib)
    if cnt>10:
        break

{http://www.mediawiki.org/xml/export-0.10/}siteinfo {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}


In [46]:
import glob
import unicodedata
import re
from bs4 import BeautifulSoup

tag="{http://www.mediawiki.org/xml/export-0.10/}page"
elems=root.findall(tag)
n_data=200
start_id=300
cnt=0

## スキップするセクションを設定する
skip_section = False  # スキップするセクションのトラッキングに使用
skip_section_headers = [
    '== 目次 ==',
    '==目次==',
    '== 外部リンク ==',
    '==外部リンク==',
    '== 参考文献 ==',
    '== 関連項目 ==',
    '===関連項目===',
    '=== 連絡先 ===',
    '=== 割と高度な内容 ===', #https://ja.wikibooks.org/wiki/C%E8%A8%80%E8%AA%9E#%E5%89%B2%E3%81%A8%E9%AB%98%E5%BA%A6%E3%81%AA%E5%86%85%E5%AE%B9
]

## 接頭辞
prefix_word = r'^[\*:#;]+\s*'

## 複数行にわたって削除する
delete_objects =[
  r'\{\|.*?\|\}', # wikitable などを削除する {| 任意の複数行のテキスト |}
  r'\{\{cite book.*?\}\}', # cite book などを削除する {{cite book }}
  r'<gallery>.*?</gallery>', # <gallery> タグを削除する
  r'<ref>.*?</ref>', # <ref> タグを削除する
]

## 行全体を削除する
delete_lines = [
  r'^\d{4}年\d{1,2}月\d{1,2}日 \(.\) \d{2}:\d{2}.*\n?', # 正規表現で日付、日時で始まる行を探して削除
  r'^<.*？>$', # html タグの行を削除する
  r'^File:.*$', # 例: File:四角形.png|thumb|1300px|* これも、しかくけい　です。
  r'^\[\[Image:.*?\]\]$', # 例: [[Image:Square (polygon).png|200px|]]
  r'^\[\[画像:.*?\]\]$', # 例: [[画像:Circle - black simple.svg|thumb|150px|left|円（えん） ]]
]

## 削除する単語（前処理）
pre_delete_words = [
  r'\{\{進捗.*?\}\}',       #{{進捗 任意のテキスト}}
  r'Wikipedia:',
  r'\{\{NDC|.*?\}\}',      # 例 {{NDC|007.64}}
  r'\[\[カテゴリ:.*?\]\]',    # 例 [[カテゴリ:アジアの言語]]
  r'\[\[Category:.*?\]\]',  # 例 [[Category:日本語|*]]
  r'\u3000',              # 全角スペース削除
  ]

## 任意のテキストを取り出す
pickup_words = [
  r'\[\[w:.*?\|(.*?)\]\]',    # [[w:不要な任意のテキスト|任意のテキスト]]
  r'\[\[.*?\|(.*?)\]\]',      # [[不要な任意のテキスト|任意のテキスト]]
  r'\[\[:.*?:.*?\:(.*?)\]\]', # [[:不要な任意のテキスト:不要な任意のテキスト:任意のテキスト]]
  r'\[\[.*?:(.*?)\]\]',      # [[不要な任意のテキスト:任意のテキスト]]
  r'\{\{[^|]*\|(.*?)\}\}',    # {{不要な任意のテキスト|任意のテキスト}}
  r"'''(.*?)'''",              # '''任意のテキスト'''
  r'\[\[(.*?)\]\]',           # [[任意のテキスト]]
  ]

## 削除する単語
delete_words = [
  r'\{\{.*?\}\}',             #{{任意のテキスト}}
  ]

# 処理されたテキストを格納するための空リスト
processed_lines = []

for elem in elems:
  cnt+=1
  if cnt<start_id:
    continue
  if cnt>n_data+start_id:
    break

  for elem2 in elem:
    for elem3 in elem2:
      if elem3 is not None and elem3.text is not None:
        xml_text = elem3.text
        soup = BeautifulSoup(xml_text, 'html.parser')
        text = soup.get_text()
        # print('-----------')
        # print(text)
        # print('-----------')

        # ページタイトルを取得したいがうまくいかない。
        # match = re.search(r'<title>(.*?)</title>', xml_text)
        # if match:
        #     # グループ1に対応するテキスト（タグ内の内容）を出力
        #     print(match.group(1))
        # else:
        #     print("タイトルタグが見つかりません。")

        ##### ページ全体に関わる処理はここで実施する
        for delete_object in delete_objects:
          text = re.sub(delete_object, '', text, flags=re.DOTALL)

        for delete_line in delete_lines:
          text= re.sub(delete_line, '', text, flags=re.MULTILINE)

        for pre_delete_word in pre_delete_words:
          text = re.sub(pre_delete_word, '', text)

        for pickup_word in pickup_words:
          text = re.sub(pickup_word, r'\1', text)

        for delete_word in delete_words:
          text = re.sub(delete_word, '', text)

        if len(text)<100:
          continue

        lines = text.split('\n')

        ##### 一行ずつ判定する処理はここで実施する
        for line in lines:
          cleaned_line = line.strip()  # 先頭と末尾の空白を削除
          # 任意のセクションヘッダーにマッチするかチェック
          is_section_header = any(cleaned_line == header for header in skip_section_headers)

          if is_section_header:
            skip_section = True  # スキップ開始
            continue

          if skip_section and cleaned_line.startswith("==") and cleaned_line.endswith("=="):
            skip_section = False  # スキップ終了
            continue

          if not skip_section:

            # セクションをスキップする
            if cleaned_line.startswith("=") and cleaned_line.endswith("="):
              continue

            # 箇条書きのリストをスキップする。「。」で終わっている場合は、文章とみなす。
            if re.match(r'^[\*:#]+\s*', line) and not line.endswith('。'):
              continue

            line = re.sub(prefix_word, '', line)
            # print(line)  # スキップフラグが立っていない行を出力
            processed_lines.append(line)

final_text = '\n'.join(processed_lines)

output_file_path = 'cleaned_text.txt'
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(final_text)

<ipython-input-46-ebc4a6c6da92>:86: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(xml_text, 'html.parser')
